In [1]:
# Import 
import os
import time
import json
import random
import datetime
import numpy as np
import pandas as pd

from IPython.display import display

from pathlib import Path
from tyssue import config
from tyssue import Sheet
from tyssue.io import hdf5
from tyssue.io.hdf5 import load_datasets
from tyssue.topology import all_rearangements
from tyssue.core.history import HistoryHdf5
from tyssue.dynamics import SheetModel as basemodel
from tyssue.solvers.quasistatic import QSSolver
from tyssue.draw import sheet_view
from tyssue.draw.plt_draw import quick_edge_draw
from tyssue.draw.ipv_draw import sheet_view as ipv_draw 

from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.apoptosis_events import apoptosis
from tyssue.behaviors.sheet.basic_events import reconnect

import matplotlib.pyplot as plt
%matplotlib inline

try:
    import seaborn as sns
except ImportError:
    print('Seaborn not found')



In [2]:
from polarity.dynamics import model
from polarity.apoptosis import apoptosis
from polarity.dynamics import EllipsoidLameGeometry as geom
from polarity.toolbox import (init,
                              define_fold_position,
                              apoptosis_ventral,
                              apoptosis_lateral,
                              apoptosis_dorsal,
                             )


In [3]:
SIM_DIR = Path('/home/admin-suz/Documents/Simulations/test_history')

sim_save_dir = SIM_DIR

try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

In [4]:
solver = QSSolver(with_t1=False, with_t3=False, with_collisions=False)

In [5]:
from tyssue.dynamics import units, effectors, model_factory
from polarity.dynamics import RadialTension
model = model_factory(
    [
        RadialTension,
        effectors.LineTension,
        #effectors.FaceContractility,
        effectors.FaceAreaElasticity,
        effectors.LumenVolumeElasticity,
    ], effectors.FaceAreaElasticity)

In [6]:
# Create tissue tissue 

def load_before_apopto(anchor_tension=0):

    #h5store = '../examples/superegg_final.hf5'
    h5store = '../examples/tissue_prepolarized.hf5'

    datasets = load_datasets(h5store,
                             data_names=['face', 'vert', 'edge'])
    
    with open('../examples/superegg_final.json', 'r+') as fp:
        specs = json.load(fp)
    
    sheet = Sheet('spherical', datasets, specs)
    sheet.settings['geometry'] = "spherical"
    sheet.settings['lumen_vol_elasticity'] = 3.e-6
    #sheet.settings['lumen_prefered_vol'] = sheet.settings['lumen_prefered_vol'] - sheet.settings['lumen_prefered_vol']*10/100

    return sheet

sheet = load_before_apopto(0)

geom.update_all(sheet)



In [7]:
sheet.edge_df.line_tension *= 40.
sheet.edge_df.line_tension.unique()
sheet.face_df["radial_tension"]=0

In [15]:
res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-8})
res


      fun: 1072.720805040367
 hess_inv: <9900x9900 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 4.29818459e-05, -4.86494288e-05, -3.75382520e-05, ...,
       -7.10298450e-05, -2.31379500e-05,  4.09387675e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 126
      nit: 86
   status: 0
  success: True
        x: array([ -7.24738705,  51.7822252 , -16.33251124, ...,  41.97969564,
        20.87542414, -74.85127254])

In [9]:
"""import ipyvolume as ipv
ipv.clear()
# code couleur profondeur du tissu
fig, mesh = sheet_view(sheet, mode="3D", coords=['z', 'x', 'y'],
                   edge={'color':sheet.vert_df.y*(sheet.vert_df.y>0),
                         'colormap':'viridis'})
fig"""
import ipyvolume as ipv
ipv.clear()
# code couleur profondeur du tissu
fig, mesh = sheet_view(sheet, mode="3D", coords=['z', 'x', 'y'],
                   edge={'color':'white'},
                      face={'color':'black', 'visible':True})
fig



Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0…

In [10]:
define_fold_position(sheet, fold_number = 1, position=[-8, 8])
print(len(sheet.face_df[sheet.face_df.fold==1]))

ipv.clear()
# code couleur profondeur du tissu
fig, mesh = sheet_view(sheet, mode="3D", coords=['z', 'x', 'y'],
                   edge={'color':'lightslategrey',
                         'visible':True},
                   face={'color': sheet.face_df.fold,
                        'colormap':'Reds',
                        'visible':True})
fig


112


Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0…

In [11]:
sheet.face_df['apoptosis'] = 0
apoptosis_ventral(sheet, 1)
apoptosis_lateral(sheet, 1)
apoptosis_dorsal(sheet, 1)

ipv.clear()
# code couleur profondeur du tissu
fig, mesh = sheet_view(sheet, mode="3D", coords=['z', 'x', 'y'],
                   edge={'color':'lightslategrey',
                         'visible':True},
                   face={'color': sheet.face_df.apoptosis,
                        'colormap':'Reds',
                        'visible':True})
fig

print(len(sheet.face_df[sheet.face_df.fold>0]))
print(len(sheet.face_df[sheet.face_df.apoptosis>0]))
print(len(sheet.face_df[sheet.face_df.apoptosis==1]))
print(len(sheet.face_df[sheet.face_df.apoptosis==2]))
print(len(sheet.face_df[sheet.face_df.apoptosis==3]))

112
36
17
14
5


In [12]:
def run_sim(sim_save_dir, sheet, constriction, radial_tension, iteration=0):
    
    dirname = '{}_constriction_{}_radialtension{}'.format(
        constriction,radial_tension, iteration)
    dirname = os.path.join(sim_save_dir, dirname)
    
    print('starting {}'.format(dirname))
    try:
        os.mkdir(dirname)
    except IOError:
        pass

    
    apoptosis_settings = {
        "shrink_rate": 1.1,
        "critical_area": 5.,
        "contract_rate": constriction,
        "basal_contract_rate": 1.01,
        "contract_span": 4,
        "radial_tension": radial_tension,
        "max_traction":30.,
        "current_traction":0.,
        "geom": geom,
    }
    
    
    # Add some information to the sheet and copy initial sheet
    sheet2 = sheet.copy(deep_copy=True)
    sheet2.face_df['id'] = sheet2.face_df.index.values
    sheet2.settings['apoptosis'] = apoptosis_settings
    
    #""" Initiale find minimal energy
    # To be sure we are at the equilibrium
    res = solver.find_energy_min(sheet2, geom, model, options={"gtol": 1e-8})


    sheet2 = apoptosis_process(dirname, solver, sheet2, geom, model)
    
    print('{} done'.format(dirname))
    print('~~~~~~~~~~~~~~~~~~~~~\n')

In [16]:
def apoptosis_process(dirname, solver, sheet, geom, model):
    
    # Initiate manager
    manager = EventManager('face')
    
    # Initiate history
    history = HistoryHdf5(sheet, 
                          extra_cols={"face": sheet.face_df.columns,
                                      "edge": list(sheet.edge_df.columns),
                                      "vert": list(sheet.vert_df.columns)},
                         hf5file=os.path.join(dirname, 'out.hf5'))
    
    #save settings
    pd.Series(sheet.settings).to_csv(os.path.join(dirname, 'settings.csv'))

    t = 0.
    stop = 150.
    
    # Add events to limit rosette 
    rosette_kwargs={
        'threshold_length':1e-6,
        'p_4':0.1,
        'p_5p':1e-2}
    manager.append(reconnect, **rosette_kwargs)

        
    # Run simulation
    while t < stop:
        
        # Définir sur combien de pas de temps on veut que les cellules entre en apoptose...
        # Répartir un "pointeur" qui part du ventral et va en dorsal...
        # Déclenche les cellules apoptotiques au fur et à mesure...
        
        ## Partie à tester....
        time_of_last_apoptosis = 30.   
            
        phi_min = -t*max(np.abs(sheet.face_df.phi))/30+max(np.abs(sheet.face_df.phi))

        l_index_apoptosis_cell = sheet.face_df[(np.abs(sheet.face_df.phi)>phi_min) 
                                               & (sheet.face_df.apoptosis>0)
                                              ].index.values
        apopto_kwargs = sheet.settings['apoptosis'].copy()
        for c in l_index_apoptosis_cell:
            apopto_kwargs.update(
            {
                'face_id': c,
            }
            )
            manager.append(apoptosis, **apopto_kwargs)  

        
        if t%10 == 0:
            sheet.settings['lumen_prefered_vol'] *= 0.99        
        
        # Reset radial tension at each time step
        sheet.vert_df.radial_tension = 0.
        
        
        manager.execute(sheet)
        res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-8})
        print(t)
        print(res)
        #all_rearangements(sheet)
        #res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-8})

        # add noise on vertex position to avoid local minimal.
        sheet.vert_df[
            ['x', 'y']] += np.random.normal(scale=1e-3, size=(sheet.Nv, 2))
        geom.update_all(sheet)
        
    
        
        # Save result in each time step.
        figname = os.path.join(
            dirname, 'invagination_{:04.0f}.png'.format(t))
        hdfname = figname[:-3] + 'hf5'        
        hdf5.save_datasets(hdfname, sheet)

        history.record()

        manager.update()
        t += 1

    return sheet


In [17]:
# Sequential Execution

from datetime import datetime

global_start=datetime.now()
print ("start : " + str(global_start))

radial_tension = [30]
#for rd in radial_tension:
contractility_percent = [8]
contractility_rate = [1+c/100 for c in contractility_percent]

for c in contractility_rate:
    for rd in radial_tension:
        for iteration in range (0, 1):
            run_sim(sim_save_dir, sheet, c, rd, iteration)
        
        
        
global_end = datetime.now()
print ("end : " + str(global_end))
print ('Duree totale d execution : \n\t\t')
print (global_end-global_start)

start : 2019-08-30 10:33:08.873827
starting /home/admin-suz/Documents/Simulations/test_history/1.08_constriction_30_radialtension0
0.0
      fun: 1068.6243846567213
 hess_inv: <9900x9900 LbfgsInvHessProduct with dtype=float64>
      jac: array([-3.04376731e-05,  2.11780844e-04,  4.06401736e-05, ...,
        3.89674460e-05,  1.13899398e-05,  4.63816318e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 99
      nit: 59
   status: 0
  success: True
        x: array([ -7.23025032,  51.61756804, -16.33503216, ...,  41.81152508,
        20.80305187, -74.81363321])
1.0
      fun: 1068.6237874483704
 hess_inv: <9900x9900 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.67111833e-05,  2.46491030e-04,  3.55758203e-05, ...,
        1.79872906e-05, -3.64883579e-05,  1.06829499e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 32
      nit: 14
   status: 0
  success: True
        x: array([ -7.22951506,  51.61758811, -16.33566806

KeyboardInterrupt: 

In [ ]:
history = HistoryHdf5.from_archive(os.path.join(sim_save_dir, '1.08_constriction_20_radialtension0/out.hf5'))

In [ ]:
sheet_ = history.retrieve(1.)

In [ ]:
sheet_.face_df